<a href="https://colab.research.google.com/github/jbenasuli/final_project/blob/database/dev/database/Amazon_Vine-PySpark-ETLs/PRELIM_ETL_videos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
# Find the latest version of spark 2.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.1.1'
spark_version = 'spark-3.1.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:10 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu 

In [2]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2021-05-02 23:05:00--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  --.-KB/s    in 0.1s    

2021-05-02 23:05:01 (6.56 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("BigData-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [4]:
import datetime
from pyspark.sql.functions import to_date
from pyspark.sql.functions import col
from pyspark.sql.types import StructField, StringType, IntegerType, StructType, BooleanType, DateType
from pyspark import SparkFiles

### Load Amazon Data into Spark DataFrame

Note: Enter URL for Desired Dataset

In [5]:
#1 load product segment
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Video_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   49033728|R1P1G5KZ05H6RD|6302503213|     748506413|The Night They Sa...|           Video|          5|            0|          0|   N|                Y|    Very satisfied!!|Fast shipping. Pl...| 2015-08-31|
|         US|   17857748|R106N066IUN8ZV|B000059PET|     478710180|Hamlet / Kline, N...|           Video|          5|    

### Create DataFrame - Perform Preliminary Cleaning

In [6]:
#1 Check the schema, print row & column count
df.printSchema()
print((df.count(), len(df.columns)))

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: string (nullable = true)

(380604, 15)


In [7]:
#2 Drop the round 1 columns
columns_to_drop = ['marketplace', 'product_parent', 'vine', 'review_headline', 'review_body', 'review_date']
df_dropped = df.drop(*columns_to_drop)
df_dropped.show()

+-----------+--------------+----------+--------------------+----------------+-----------+-------------+-----------+-----------------+
|customer_id|     review_id|product_id|       product_title|product_category|star_rating|helpful_votes|total_votes|verified_purchase|
+-----------+--------------+----------+--------------------+----------------+-----------+-------------+-----------+-----------------+
|   49033728|R1P1G5KZ05H6RD|6302503213|The Night They Sa...|           Video|          5|            0|          0|                Y|
|   17857748|R106N066IUN8ZV|B000059PET|Hamlet / Kline, N...|           Video|          5|            0|          0|                Y|
|   25551507| R7WTAA1S5O7D9|0788812807|Nascar Dual Power...|           Video|          4|            0|          0|                Y|
|   21025041|R32HFMVWLYOYJK|6302509939|The Man From U.N....|           Video|          5|            0|          0|                Y|
|   40943563| RWT3H6HBVAL6G|B00JENS2BI|Playboy Video Par...|  

In [8]:
#3 Filter Step 1:
df_filtered = df_dropped.filter(df_dropped.verified_purchase == 'Y')
df_filtered.show
print((df_filtered.count(), len(df_filtered.columns)))

(95732, 9)


In [9]:
#4 drop filtered verified_purchase column
columns_to_drop = ['verified_purchase']
df_dropped_2 = df_filtered.drop(*columns_to_drop)
df_dropped_2.show()

+-----------+--------------+----------+--------------------+----------------+-----------+-------------+-----------+
|customer_id|     review_id|product_id|       product_title|product_category|star_rating|helpful_votes|total_votes|
+-----------+--------------+----------+--------------------+----------------+-----------+-------------+-----------+
|   49033728|R1P1G5KZ05H6RD|6302503213|The Night They Sa...|           Video|          5|            0|          0|
|   17857748|R106N066IUN8ZV|B000059PET|Hamlet / Kline, N...|           Video|          5|            0|          0|
|   25551507| R7WTAA1S5O7D9|0788812807|Nascar Dual Power...|           Video|          4|            0|          0|
|   21025041|R32HFMVWLYOYJK|6302509939|The Man From U.N....|           Video|          5|            0|          0|
|   17013969|R1S3T3GWUGQTW7|6305761302|Cabaret Balkan - ...|           Video|          5|            0|          0|
|   47611685|R3R0QYHA800REE|6300157555|Wrinkles:in Need ...|           V

## Create Analysis-Specific DFs/Tables
### Perform Analysis-Specific Transforms

### Segmentation Analysis DF

In [10]:
# 1 Create Segmentation DF by Droppig Addtional Columns
segmentation_cols_drop = ['review_id', 'product_id', 'product_title', 'star_rating', 'helpful_votes', 'total_votes']
segmentation_dropped_df = df_dropped_2.drop(*segmentation_cols_drop)
segmentation_dropped_df.show()

+-----------+----------------+
|customer_id|product_category|
+-----------+----------------+
|   49033728|           Video|
|   17857748|           Video|
|   25551507|           Video|
|   21025041|           Video|
|   17013969|           Video|
|   47611685|           Video|
|   10747909|           Video|
|     126341|           Video|
|   40676812|           Video|
|   19863533|           Video|
|    1989554|           Video|
|   14885930|           Video|
|   24203443|           Video|
|   14885930|           Video|
|   41133700|           Video|
|   46580399|           Video|
|   19863533|           Video|
|   25551507|           Video|
|   25016862|           Video|
|    1989554|           Video|
+-----------+----------------+
only showing top 20 rows



Note: must change Category Label name in withColumnRenamed('count(product_category)', 'name')

In [11]:
#2 Segmentation GroupBy
#2a GroupBy customer_id
#2b Count product_category and rename count columns as Segment Name
segment_df = segmentation_dropped_df.groupby("customer_id")\
.agg({'product_category':'count'}).withColumnRenamed('count(product_category)', 'videos')
#2c Check results
segment_df.show()

+-----------+------+
|customer_id|videos|
+-----------+------+
|   50851028|     1|
|   20686161|     2|
|   15191478|     2|
|    2202715|     1|
|   48333068|     1|
|   22910908|     2|
|   38463562|     1|
|   26902200|     1|
|   33236947|     1|
|   40127650|     1|
|   41244127|     3|
|   22968117|     1|
|   25472255|     1|
|    3160356|     1|
|   18785199|     2|
|   19840529|     1|
|    1655605|     1|
|   26298325|     1|
|   25705446|     1|
|   29337875|     1|
+-----------+------+
only showing top 20 rows



In [12]:
#3 Check segment_df Schema and Row Count
segment_df.printSchema()
print(segment_df.count())

root
 |-- customer_id: integer (nullable = true)
 |-- videos: long (nullable = false)

65517


Note: Column Name in df.sort('name' ...) must align with Column name from step 2

In [13]:
#4 Filter for Top n Results
#4a Declare number of rows to filter by (100,000)
row_count = 100000
#4a Sort by Segment Desc and limit to row_count
filtered_segment_df = segment_df.sort("videos", ascending=False).limit(row_count)
#4b Check Results
filtered_segment_df.show()
print(filtered_segment_df.count())

+-----------+------+
|customer_id|videos|
+-----------+------+
|   18116317|   286|
|   43022286|   272|
|   49778371|   138|
|   43921586|   136|
|   14315565|   118|
|   52287429|    96|
|   49355567|    91|
|   51010646|    84|
|   15536614|    82|
|   38724344|    82|
|   45087308|    81|
|   24833576|    76|
|   28169415|    59|
|   34736206|    59|
|   53089838|    57|
|   50010538|    54|
|   12200981|    53|
|   47370694|    52|
|   44038210|    52|
|   49422747|    52|
+-----------+------+
only showing top 20 rows

65517


### Segmentation ETL Complete - Add Database Export Code

In [14]:
# Configure settings for RDS
mode = "append"
jdbc_url='jdbc:postgresql://<connection_srtring>'
config = {"user":'<username>', 
          "password": '<password>', 
          "driver":"org.postgresql.Driver"}

Note: table name in table='name_segment' must align with table name in Postgres

In [15]:
# Write segment table to Postgres/RDS
# 15 secs
filtered_segment_df.write.jdbc(url=jdbc_url, table='videos_segment', mode=mode, properties=config)

## Apriori Analysis DF
### Enter Apriori Transfomations Below - Use df_dropped_2 as Start Point

In [16]:
# 1 Create Apriori DF by First Dropping Addtional Columns
apriori_cols_drop = ['product_category', 'product_title', 'star_rating', 'helpful_votes', 'total_votes']
apriori_dropped_df = df_dropped_2.drop(*apriori_cols_drop)
apriori_dropped_df.show()

+-----------+--------------+----------+
|customer_id|     review_id|product_id|
+-----------+--------------+----------+
|   49033728|R1P1G5KZ05H6RD|6302503213|
|   17857748|R106N066IUN8ZV|B000059PET|
|   25551507| R7WTAA1S5O7D9|0788812807|
|   21025041|R32HFMVWLYOYJK|6302509939|
|   17013969|R1S3T3GWUGQTW7|6305761302|
|   47611685|R3R0QYHA800REE|6300157555|
|   10747909| RGORN81H45NI7|B000SXQ5US|
|     126341|R1CNYN4ABNOJSU|B00008F22G|
|   40676812|R2DW06821PMN40|6303453961|
|   19863533|R1CS8AMA8B0VBJ|6302453178|
|    1989554| R3XP0G8P2BOTP|B000XPQ18W|
|   14885930|R3NK7014K996PF|0800113055|
|   24203443| ROIQI6ZW6UVL9|6300133338|
|   14885930| RECPIN4UZI76Z|6304119054|
|   41133700|R3FE5KO613PMVI|B000006CPD|
|   46580399|R2EC74PS45RAOR|189218608X|
|   19863533|R3FZ5C78PP7ZD7|6302754348|
|   25551507|R2Z9SKOUI9C9IR|6302320402|
|   25016862|R37DDS6DLZTK60|B000EQJFWY|
|    1989554|R3NPKF41EJ8U44|6301801601|
+-----------+--------------+----------+
only showing top 20 rows



### Prelim Apriori ETL Complete - Add Database Export Code

In [17]:
# Configure settings for RDS
mode = "append"
jdbc_url='jdbc:postgresql://<connection_srtring>'
config = {"user":'<username>', 
          "password": '<password>', 
          "driver":"org.postgresql.Driver"}

Note: table name in table='segment_apriori' must align with table name in Postgres

In [18]:
# Write segment_apriori table to RDS
# 16 secs
apriori_dropped_df.write.jdbc(url=jdbc_url, table='videos_apriori', mode=mode, properties=config)

## Amazon Reviews S3 -> Postgres/RDS ETL Complete



### Run Queries in Postgres to Confirm Segment ETL
Check Row Count of Segment Table - SELECT COUNT(*) FROM videos_segment;

Check 10 Rows of Segment Table - SELECT * FROM videos_segment LIMIT(10);


In [19]:
# Upon Confirmation of Above Checks Run This Cell
print('Segment ETL Successful')

Segment ETL Successful


### Run Queries in Postgres to Confirm Apriori ETL
Check Row Count of Apriori Table - SELECT COUNT(*) FROM videos_apriori;

Check 10 Rows of Apriori Table - SELECT * FROM videos_apriori LIMIT(10);

In [20]:
# Upon Confirmation of Above Checks Run This Cell
print('Apriori ETL Successful')

Apriori ETL Successful
